In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from cycler import cycler
import tensorflow as tf
import seaborn as sns
import pandas
import datetime
from imdb import IMDb

In [ ]:
color_cycle_wong = ['#000000','#E69F00','#CC79A7','#56B4E9','#009E73','#0072B2','#F0E442','#D55E00']
Six_color_cycle = ["#5790FC", "#F89C20", "#E42536", "#964A8B", "#9C9CA1", "#7A21DD"]
Eight_color_cycle = ["#1845FB", "#FF5E02", "#C91F16", "#C849A9", "#ADAD7D", "#86C8DD", "#578DFF", "#656364"]
Ten_color_cycle = ["#3F90DA", "#FFA90E", "#BD1F01", "#94A4A2", "#832DB6", "#A96B59", "#E76300", "#B9AC70", "#717581", "#92DADD"]
mpl.rcParams['axes.prop_cycle'] = cycler(color=Six_color_cycle)

In [ ]:
movies = pandas.read_csv('./Misc_Files/movies.csv')
votes = pandas.read_csv('./Misc_Files/results.csv')

In [ ]:
people = np.loadtxt('./Movie_Surveys/people.txt',delimiter=',',dtype=str)
print(people)
people[7]

In [ ]:
survey_results = {}
for person in people:
    survey_results[person] = pandas.read_csv('./Movie_Surveys/survey_{}_results.csv'.format(person))
#survey_results = pandas.read_csv('./Movie_Surveys/survey_{}_results.csv'.format(people[7]))

In [ ]:
error_movie_names = ['Whiplash (2015)','28 Days Later (2002)','Hotel Rwanda (2005)',
                     'Dr. Sleep (2019)','Green Room (2016)','Chinatown (1975)','Akira (1998)',
                     'The Godfather Pt. III (1990)','Blood Fest (2019)','Get Out (2016)']
corrected_movie_names = ['Whiplash (2014)','28 Days Later... (2002)','Hotel Rwanda (2004)',
                         'Doctor Sleep (2019)','Green Room (2015)','Chinatown (1974)','Akira (1988)',
                        'The Godfather: Part III (1990)','Blood Fest (2018)','Get Out (2017)']

In [ ]:
def get_movie_info(movie_name):
    if movie_name in error_movie_names:
        movie_name = corrected_movie_names[error_movie_names.index(movie_name)]
    print(movie_name)
    search_movie = ia.search_movie(movie_name)
    if search_movie[0]['kind'] in ['movie','video movie'] or movie_name in ['Kung Fury (2015)','Lorena, Light-footed Woman (2019)']:
        imdb_movie = ia.get_movie(search_movie[0].movieID)
    else:
        i=1
        try:
            while search_movie[i]['kind'] != 'movie':
                print('next')
                i+=1
            print(i)
            imdb_movie = ia.get_movie(search_movie[i].movieID)
            print(imdb_movie['title'])
        except:
            print('')
            print('Cant find movie',movie_name)
            print('')
            imdb_movie=movie_name
    print('done.')
    return imdb_movie

In [ ]:
# create an instance of the IMDb class
ia = IMDb()

In [ ]:
perp=get_movie_info('Space Cop (2016)')
perp['genre']

In [ ]:
tmp = ia.search_movie('Space Cop (2016)')[0]
burp = ia.get_movie(tmp.movieID)
print(burp.current_info)
print(burp['genre'])
print(burp.keys())
print(burp['kind'])

In [ ]:
survey_results.keys()

In [ ]:
survey_results_dict = {}
for name,survey_result in survey_results.items():
    skip=True
    for col, val in survey_result.items():
        if isinstance(col,str) and not skip:
            count = int(col)
            if count%2 == 0:
                for i,movie_name in enumerate(val):
                    if survey_result['{}'.format(count+1)][i] != 0:
                        if movie_name not in survey_results_dict.keys():
                            survey_results_dict[movie_name] = {'votes':int(survey_result['{}'.format(count+1)][i]),
                                                               'imdb_movie':get_movie_info(movie_name)}
                        else:
                            survey_results_dict[movie_name]['votes'] += int(survey_result['{}'.format(count+1)][i])
        skip=False
    print('------------')
    print(name,'done.')
    print('------------')

In [ ]:
tmp_votes= {}
for key in survey_results_dict.keys():
    if survey_results_dict[key]['votes'] > 5:
        tmp_votes[key] = survey_results_dict[key]['votes']
    
most_voted = []
for i,(mov,votes) in enumerate(sorted(tmp_votes.items(), key=lambda x: x[1], reverse=True)):
    if i < 10:
        gs = survey_results_dict[mov]['imdb_movie']['genres']
        gs_len = np.floor(votes/len(gs))
        sht_arr = [mov,votes]
        for j,g in enumerate(gs):
            if j == 0:
                sht_arr += [g,gs_len+votes%len(gs)]
            else:
                sht_arr += [g,gs_len+sht_arr[len(sht_arr)-1]]
        most_voted.append(sht_arr)
print(most_voted)
df_most_voted = pandas.DataFrame(data=most_voted,
                                 columns=['Movies','Votes','Main Genre','div0',
                                          'Subgenre 1','div1','Subgenre 2','div2','Subgenre 3','div3',
                                          'Subgenre 4','div4','Subgenre 5','div5'])#,'Subgenre 6','div6'])
df_most_voted

In [ ]:
genres = {}
for movie in survey_results_dict.keys():
    if not isinstance(survey_results_dict[movie]['imdb_movie'],str):
        for genre in survey_results_dict[movie]['imdb_movie']['genre']:
            if genre not in genres:
                genres[genre] = 1
            else:
                genres[genre] += 1

sorted_genres = []
for s_g,votes in sorted(genres.items(), key=lambda x: x[1], reverse=True):
    sorted_genres.append([s_g,votes])
df_sorted_genres = pandas.DataFrame(data=sorted_genres,columns=['Genres','Votes'])

In [ ]:
print(sorted_genres)
for i,sub in enumerate(sorted_genres):
    if 'Drama' in sub:
        print(sorted_genres[i][0])
print([i for i,sub in enumerate(sorted_genres) if 'Crime' in sub])

In [ ]:
plt.figure(figsize=(15,10))
sns.barplot(x="Votes", y="Genres", data=df_sorted_genres,
            label="Total", color="b",)
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(15,10))
# Plot the crashes where alcohol was involved
sns.set_color_codes("pastel")
sns.barplot(x="Votes", y="Movies", data=df_most_voted,color="b")

for div in range(5,-1,-1):
    #print(df_most_voted['Main Genre'])
    if div == 0:
        for (i,d),g in zip(enumerate(df_most_voted['div{}'.format(div)]),df_most_voted['Main Genre']):
            colour = [i for i,sub in enumerate(sorted_genres) if g in sub][0]
            plt.text(d/2, i, g,  # x, y, text,
                     ha='center', va='center',   # text alignment,
                     transform=ax.transData,      # coordinate system transformation
                     fontdict={'fontsize':14,'color':'C{}'.format(colour)})
        sns.set_color_codes("muted")
        sns.barplot(x="div{}".format(div), y="Movies", data=df_most_voted,
                    label="Main Genre", color="C{}".format(div+1))
    else:
        for (i,d),g in zip(enumerate(df_most_voted['div{}'.format(div)]),df_most_voted['Subgenre {}'.format(div)]):
            text_x = df_most_voted['div{}'.format(div-1)][i]+(d - df_most_voted['div{}'.format(div-1)][i])/2
            if not np.isnan(text_x):
                colour = [i for i,sub in enumerate(sorted_genres) if g in sub][0]
                plt.text(text_x, i, g,  # x, y, text,
                         ha='center', va='center',   # text alignment,
                         transform=ax.transData,      # coordinate system transformation
                         fontdict={'fontsize':14,'color':'C{}'.format(colour)})
        sns.barplot(x="div{}".format(div), y="Movies", data=df_most_voted,
                    label="Subgenre {}".format(div), color="C{}".format(div+1))

# Add a legend and informative axis label
ax.legend(ncol=2, loc="lower right", frameon=True)
#ax.set(xlim=(0, 24), ylabel="",
#       xlabel="Automobile collisions per billion miles")
sns.despine(left=True, bottom=True)

In [ ]:
morvie = sorted_votes[0][0]
i = 0
imdb_movie = ia.search_movie(morvie)[i]
if 'movie' not in imdb_movie['kind']:
    imdb_movie = ia.search_movie(morvie)[i+1]

In [ ]:
print([blorp['kind'] for blorp in ia.search_movie(morvie)])

In [ ]:
imdb_movie.keys()

In [ ]:
imdb_movie.current_info

In [ ]:
imdb_movie['year']

# Tensor Flow Examples

In [ ]:
celsius_q    = np.array([-40, -10,  0,  8, 15, 22,  38],  dtype=float)
fahrenheit_a = np.array([-40,  14, 32, 46, 59, 72, 100],  dtype=float)
l0 = tf.keras.layers.Dense(units=1, input_shape=[1]) 
model = tf.keras.Sequential([l0])
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.1))
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
history = model.fit(celsius_q, fahrenheit_a, epochs=500, verbose=False)
model.predict([100.0])

## Transfer Learning